In [1]:
import numpy as np
from dimod import ConstrainedQuadraticModel, BinaryQuadraticModel, QuadraticModel
from dwave.system import LeapHybridCQMSampler
import pandas as pd
import urllib.request
from dotenv import load_dotenv, find_dotenv
import os

In [2]:
# Read players rating from CSV
filename = 'https://raw.githubusercontent.com/dwave-examples/Line-up-optimization/master/players.csv'
column_names = ["Variable", "Player", "Position", "Rating"]
players_df = pd.read_csv(filename, names=column_names)
ratings = players_df['Rating'].tolist()

In [3]:
players_df

,Variable,Player,Position,Rating
0,x[0],Alisson,GK,6.81
1,x[1],Adrian,GK,5.86
2,x[2],vanDijk,DC,6.62
3,x[3],Gomez,DC,6.91
4,x[4],Matip,DC,6.69
5,x[5],Phillips,DC,7.24
6,x[6],Fabinho,DC,7.11
7,x[7],Robertson,DL,6.85
8,x[8],Arnold,DR,6.93
9,x[9],Williams,DR,6.63


In [4]:
# Load Env Variables
load_dotenv(find_dotenv())
token = os.environ['DWAVE_API_KEY'] 

In [5]:
n_var = players_df.shape[0]
print("\nBuilding a CQM for 11 amongst {} players.".format(str(n_var)))

robertson = [7,11,16]
jota = [27,32,34,37,41]

ratings_robertson = [e for i, e in enumerate(ratings) if i in robertson]
ratings_jota = [e for i, e in enumerate(ratings) if i in jota]


Building a CQM for 11 amongst 43 players.


In [6]:
p_labels = [f'{i}' for i in range(n_var)]

In [10]:
cqm = ConstrainedQuadraticModel()

# Set the objective: maximime the number of best rated players 
objective = QuadraticModel()
objective.add_linear_from(
                    (
                    (p_labels[i], -ratings[i]) for i in range(n_var) 
                    ), 
                    default_vartype='BINARY'
                    )

cqm.set_objective(objective)

# Select only 11 players
constraint = QuadraticModel()
constraint.add_linear_from(
                (
                (p_labels[i], 1) for i in range(n_var) 
                ), 
                default_vartype='BINARY'
                )
cqm.add_constraint(constraint, sense="==", rhs=11, label='11 players')

# Select only 1 goalkeeper
constraint = QuadraticModel()
constraint.add_linear_from(
                (
                (p_labels[i], 1) for i in range(2) 
                ), 
                default_vartype='BINARY'
                )
cqm.add_constraint(constraint, sense="==", rhs=1, label='1 goalkeeper')

# Select only 2 Central Defender
constraint = QuadraticModel()
constraint.add_linear_from(
                (
                (p_labels[i], 1) for i in range(2,7) 
                ), 
                default_vartype='BINARY'
                )
cqm.add_constraint(constraint, sense="==", rhs=2, label='2 Central Defenders ')

# Select only 1 Defender Left
constraint = QuadraticModel()
constraint.add_linear_from(
                (
                (p_labels[i], 1) for i in range(7,8) 
                ), 
                default_vartype='BINARY'
                )
cqm.add_constraint(constraint, sense="==", rhs=1, label='1 Defender Left ')

# Select only 1 Defender Right
constraint = QuadraticModel()
constraint.add_linear_from(
                (
                (p_labels[i], 1) for i in range(8,11) 
                ), 
                default_vartype='BINARY'
                )
cqm.add_constraint(constraint, sense="==", rhs=1, label='1 Defender Right ')

# Select only 1 Forward Striker
constraint = QuadraticModel()
constraint.add_linear_from(
                (
                (p_labels[i], 1) for i in range(38,42) 
                ), 
                default_vartype='BINARY'
                )
cqm.add_constraint(constraint, sense="==", rhs=1, label='1 Forward Striker ')

# Select only 3 Midfielders
constraint = QuadraticModel()
constraint.add_linear_from(
                (
                (p_labels[i], 1) for i in range(16,28) 
                ), 
                default_vartype='BINARY'
                )
cqm.add_constraint(constraint, sense="==", rhs=3, label='3 Midfielders ')

# Select only 1 Forward Left
constraint = QuadraticModel()
constraint.add_linear_from(
                (
                (p_labels[i], 1) for i in range(33,36) 
                ), 
                default_vartype='BINARY'
                )
cqm.add_constraint(constraint, sense="==", rhs=1, label='1 Forward Left ')

# Select only 1 Forward Right
constraint = QuadraticModel()
constraint.add_linear_from(
                (
                (p_labels[i], 1) for i in range(36,38) 
                ), 
                default_vartype='BINARY'
                )
cqm.add_constraint(constraint, sense="==", rhs=1, label='1 Forward Right ')

# Select Robertson in a position
constraint = QuadraticModel()
constraint.add_linear_from(
                (
                (p_labels[i], 1) for i in robertson 
                ), 
                default_vartype='BINARY'
                )
cqm.add_constraint(constraint, sense="==", rhs=1, label='Robertson in a position ')

# Select Jota in a position
constraint = QuadraticModel()
constraint.add_linear_from(
                (
                (p_labels[i], 1) for i in jota 
                ), 
                default_vartype='BINARY'
                )
cqm.add_constraint(constraint, sense="==", rhs=1, label='Jota in a position ')


'Jota in a position '

In [11]:
print(cqm)

Constrained quadratic model: 43 variables, 11 constraints, 126 biases

Objective
  -6.81*Binary('0') - 5.86*Binary('1') - 6.62*Binary('2') - 6.91*Binary('3') - 6.69*Binary('4') - 7.24*Binary('5') - 7.11*Binary('6') - 6.85*Binary('7') - 6.93*Binary('8') - 6.63*Binary('9') - 8.15*Binary('10') - 6.97*Binary('11') - 6.4*Binary('12') - 7.66*Binary('13') - 6.64*Binary('14') - 6.8*Binary('15') - 6.64*Binary('16') - 7.77*Binary('17') - 6.58*Binary('18') - 6.74*Binary('19') - 7.38*Binary('20') - 6.97*Binary('21') - 6.43*Binary('22') - 6.7*Binary('23') - 7.13*Binary('24') - 6.49*Binary('25') - 6.05*Binary('26') - 9.39*Binary('27') - 7.16*Binary('28') - 6.52*Binary('29') - 6.8*Binary('30') - 7.24*Binary('31') - 6.77*Binary('32') - 7.56*Binary('33') - 7.23*Binary('34') - 6.4*Binary('35') - 7.42*Binary('36') - 7.84*Binary('37') - 6.99*Binary('38') - 6.8*Binary('39') - 6.03*Binary('40') - 8.22*Binary('41') - 5.84*Binary('42')

Constraints
  11 players: Binary('0') + Binary('1') + Binary('2') + Binar

In [12]:
sampler = LeapHybridCQMSampler(token=token)
print("Submitting CQM to solver {}.".format(sampler.solver.name))
sampleset = sampler.sample_cqm(cqm, label='Example - Lineup')

Submitting CQM to solver hybrid_constrained_quadratic_model_version1.


In [13]:
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
if not len(feasible_sampleset):
    raise ValueError("No feasible solution found")
best = feasible_sampleset.first
selected_item_indices = [key for key, val in best.sample.items() if val==1.0]
selected_item_indices = [eval(i) for i in selected_item_indices]

In [14]:
selected_item_indices

[0, 10, 17, 20, 27, 33, 36, 38, 5, 6, 7]

In [20]:
result =  players_df.filter(items = selected_item_indices, axis=0).drop('Variable', axis=1).sort_index().reset_index(drop=True)
result

,Player,Position,Rating
0,Alisson,GK,6.81
1,Phillips,DC,7.24
2,Fabinho,DC,7.11
3,Robertson,DL,6.85
4,Milner,DR,8.15
5,Williams,CM,7.77
6,Thiago,CM,7.38
7,Jota,CM,9.39
8,Mane,FWL,7.56
9,Salah,FWR,7.42


In [18]:
print(result['Rating'].sum())

82.67
